In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json


OPEN_API_KEY = 'openai_api_key'
EMBEDDING_MODEL = 'jhgan/ko-sroberta-multitask' #all-MiniLM-L6-v2

In [ ]:
# Sentence-transformers 사용
# 임베딩 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 문서 리스트
documents = [
    "기후 변화는 심각한 문제입니다.",
    "AI는 산업에 큰 영향을 주고 있습니다.",
    "환경 보호는 우리의 미래를 지키는 일입니다.",
    "야구는 인기 있는 스포츠입니다."
]

# 문서 임베딩
doc_embeddings = model.encode(documents, convert_to_numpy=True)

# FAISS 인덱스 생성
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

# 쿼리 벡터 임베딩
query = "환경 보호에 관련된 내용이 뭐야?"
query_vector = model.encode([query], convert_to_numpy=True)

# 유사 문서 검색
_, indices = index.search(query_vector, k=1)
print(f"질문: {query}")
print(f"가장 유사한 문장: {documents[indices[0][0]]}")


In [ ]:
# LangChain 사용
# 문서 리스트
documents = [
    "기후 변화는 심각한 문제입니다.",
    "AI는 산업에 큰 영향을 주고 있습니다.",
    "환경 보호는 우리의 미래를 지키는 일입니다.",
    "야구는 인기 있는 스포츠입니다."
]

# 임베딩 + 벡터저장소(스토어) 생성
embedding = HuggingFaceEmbeddings(model_name=f"{EMBEDDING_MODEL}")
db = FAISS.from_texts(texts=documents, embedding=embedding)

# 질문에 대한 유사 문서 검색
query = "환경 보호에 관련된 내용이 뭐야?"
results = db.similarity_search(query, k=1)

print(f"질문: {query}")
print(f"가장 유사한 문장: {results[0].page_content}")


In [ ]:
# JSON 문서 로드
with open('docs.json', encoding='utf-8') as f:
    raw_data = json.load(f)

documents = [d["text"] for d in raw_data]

# 문서 전처리
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
docs = splitter.create_documents(documents)

# 한국어 임베딩 모델
embedding_model = HuggingFaceEmbeddings(model_name=f"{EMBEDDING_MODEL}")

# 벡터저장소(스토어) 생성
db = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./chroma_db")
db.persist()

# OpenAI 모델 로드
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=f"{OPEN_API_KEY}")

# RAG QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(search_type="similarity", k=3),
    return_source_documents=True
)

query = "개인정보 수집 시 필요한 법적 조건은?"
result = qa_chain(query)

print("답변:", result['result'])
print("출처 문서:", result['source_documents'])
